#### Install needed packages

In [1]:
%pip install --upgrade --quiet langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


#### Check that required API keys are available in environment variables

In [2]:
import os
if not os.environ.get('OPENAI_API_KEY'): print('Warning: OPENAI_API_KEY not found')

#### Define simple chain with prompt and LLM

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

LLM_MODEL = 'gpt-4o'

llm = ChatOpenAI(model=LLM_MODEL)

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('user', '{input}')
])
simple_chat_chain = prompt | llm 


#### Content streamer

In [4]:
async def generate_chat_events(message):
    async for evt in simple_chat_chain.astream_events(message, version='v1'):
        if evt['event'] == 'on_chain_stream':
            chunk = evt['data']['chunk'].content if evt.get('data').get('chunk') else None
            if chunk:
                yield chunk

#### Try it out

In [5]:
prompt = 'What is the capital of France?'
async for event in generate_chat_events({'input': prompt, 'chat_history': []}):
    print(event, end='')

The capital of France is Paris.